<a href="https://colab.research.google.com/github/DrJHSIM/DeepLearningArticles/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2024_3_29 : AlexNet Torch Model에서 가지고와 테스트로 돌리기.
2024_3_30 : AlexNet 논문과 몇 가지 다른 사람이 구현한 예제를 가지고 비교하며 수정.

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import torch.optim as optim
import torch.nn.functional as F

import torchvision.models as models

In [9]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.RandomHorizontalFlip(p=0.5),
    ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform = transform)


# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
for x, y in training_data:
    print(x.shape)
    break

torch.Size([3, 227, 227])


https://junha1125.github.io/blog/pytorch-docker-git/2020-08-03-CIFAR10_TransferLearning/

https://medium.com/nerd-for-tech/classification-on-cifar-10-32abe456302

https://wikidocs.net/49588

In [11]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [12]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


https://pytorch.org/vision/stable/_modules/torchvision/models/alexnet.html#alexnet

In [20]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 10, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            #conv1
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            #conv2
            nn.Conv2d(96, 256, kernel_size=5, stride = 1, padding = 2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            #conv3
            nn.Conv2d(256, 384, kernel_size=3, stride = 1, padding = 1),
            nn.ReLU(inplace=True),
            #conv4
            nn.Conv2d(384, 384, kernel_size=3, stride = 1, padding = 1),
            nn.ReLU(inplace=True),
            #conv5
            nn.Conv2d(384, 256, kernel_size=3, stride = 1, padding = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            #fc6
            nn.Dropout(p=dropout),
            nn.Linear(256*6*6, 4096),
            nn.ReLU(inplace=True),
            #fc7
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            #fc8
            nn.Linear(4096, num_classes),
        )

    # bias, weight 초기화
    def init_bias_weights(self):
      for layer in self.features:
        if isinstance(layer, nn.Conv2d):
          nn.init.normal_(layer.weight, mean=0, std=0.01)   # weight 초기화
          nn.init.constant_(layer.bias, 0)   # bias 초기화
      # conv 2, 4, 5는 bias 1로 초기화
      nn.init.constant_(self.features[4].bias, 1)
      nn.init.constant_(self.features[10].bias, 1)
      nn.init.constant_(self.features[12].bias, 1)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = torch.flatten(x, 1)
        logit = self.classifier(x)
        return logit


output 계산 :
https://dingyan89.medium.com/calculating-parameters-of-convolutional-and-fully-connected-layers-with-keras-186590df36c6

In [21]:
model = AlexNet().to(device)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): L

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

ToTensor()와 Resize위치 바꿈에 따라 성능 차이가 있음 (ToTensor()가 먼저 나오면 정확도 10%에서 멈춤)

In [26]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.378446  [   64/50000]
loss: 1.331109  [ 6464/50000]
loss: 1.084223  [12864/50000]
loss: 1.266245  [19264/50000]
loss: 1.109479  [25664/50000]
loss: 1.441420  [32064/50000]
loss: 1.110459  [38464/50000]
loss: 1.380409  [44864/50000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.214737 

Epoch 2
-------------------------------
loss: 1.214322  [   64/50000]
loss: 1.320377  [ 6464/50000]
loss: 1.009027  [12864/50000]
loss: 1.271639  [19264/50000]
loss: 1.077908  [25664/50000]
loss: 1.444918  [32064/50000]
loss: 1.114537  [38464/50000]
loss: 1.212335  [44864/50000]
Test Error: 
 Accuracy: 63.3%, Avg loss: 1.038045 

Epoch 3
-------------------------------
loss: 1.138170  [   64/50000]
loss: 1.166607  [ 6464/50000]
loss: 1.033029  [12864/50000]
loss: 1.137864  [19264/50000]
loss: 0.960621  [25664/50000]
loss: 1.154229  [32064/50000]
loss: 1.021966  [38464/50000]
loss: 1.136142  [44864/50000]
Test Error: 
 Accuracy: 64.7%, Avg loss: 1.025180 

Epoc

(2024-4-10 : initialize 중요하다)

(2024/4/9)정확도가 향상되지 않으므로 loss_fn, optimizer 등을 조정하여 향상시켜볼 것.

In [27]:
torch.save(model.state_dict(), "AlexNet_2024_4_26.pth")
print("Saved PyTorch Model State to AlexNet_2024_4_26.pth")

Saved PyTorch Model State to AlexNet_2024_4_26.pth


In [28]:
model = AlexNet().to(device)
model.load_state_dict(torch.load("AlexNet_2024_4_26.pth"))

<All keys matched successfully>

In [43]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]


model.eval()
for x, y in test_dataloader:
    x , y = x.to(device), y.to(device)
    pred = model(x)
    correct = (pred.argmax(axis = 1) == y).type(torch.float).sum()
    accuracy = correct/len(x)*100
    print(f"accuracy: {accuracy:>0.1f}%")
    break

accuracy: 68.8%
